## Import

In [49]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Download data

In [6]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554 entries, 0 to 1553
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Age        1554 non-null   int64
 1   Salary     1554 non-null   int64
 2   Purchased  1554 non-null   int64
dtypes: int64(3)
memory usage: 36.5 KB


In [8]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


In [9]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


## Data preparation

In [10]:
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Model preparation

In [13]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(2, activation='softmax'),
])

In [14]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Model fit

In [15]:
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
39/39 [==============================] - 1s 2ms/step - loss: 0.7123 - accuracy: 0.4903
Epoch 2/50
39/39 [==============================] - 0s 1ms/step - loss: 0.4909 - accuracy: 0.8599
Epoch 3/50
39/39 [==============================] - 0s 1ms/step - loss: 0.3708 - accuracy: 0.9121
Epoch 4/50
39/39 [==============================] - 0s 1ms/step - loss: 0.2830 - accuracy: 0.9190
Epoch 5/50
39/39 [==============================] - 0s 1ms/step - loss: 0.2341 - accuracy: 0.9273
Epoch 6/50
39/39 [==============================] - 0s 1ms/step - loss: 0.2027 - accuracy: 0.9224
Epoch 7/50
39/39 [==============================] - 0s 1ms/step - loss: 0.1863 - accuracy: 0.9252
Epoch 8/50
39/39 [==============================] - 0s 1ms/step - loss: 0.1900 - accuracy: 0.9330
Epoch 9/50
39/39 [==============================] - 0s 1ms/step - loss: 0.1569 - accuracy: 0.9496
Epoch 10/50
39/39 [==============================] - 0s 1ms/step - loss: 0.1618 - accuracy: 0.9417
Epoch 11/50
39/39 [

In [17]:
loss, accuracy = model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 2ms/step - loss: 0.1173 - accuracy: 0.9550


In [18]:
loss, accuracy

(0.11725999414920807, 0.9549839496612549)

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                30        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 162
Trainable params: 162
Non-trainable params: 0
_________________________________________________________________


## Test model

In [38]:
prediction_1 = model.predict(sc.transform(np.array([[42, 50000]])))
prediction_1, np.argmax(prediction_1)

(array([[0.3051851 , 0.69481486]], dtype=float32), 1)

In [40]:
prediction_2 = model.predict(sc.transform(np.array([[20, 40000]])))
prediction_2, np.argmax(prediction_2)

(array([[0.994369  , 0.00563109]], dtype=float32), 0)

Save model to folder

In [41]:
model.save('customer_behavior_model/1')

INFO:tensorflow:Assets written to: customer_behavior_model/1/assets


In [42]:
!ls

customer_behavior_model  sample_data


In [43]:
!ls customer_behavior_model/1

assets	saved_model.pb	variables


Check saved model

In [44]:
from tensorflow.keras.models import load_model

In [45]:
cust_model = load_model('customer_behavior_model/1')

In [46]:
prediction_3 = cust_model.predict(sc.transform(np.array([[42, 50000]])))
prediction_3, np.argmax(prediction_3)

(array([[0.3051851 , 0.69481486]], dtype=float32), 1)

## Model download from colab

In [47]:
!zip -r customer.zip customer_behavior_model

  adding: customer_behavior_model/ (stored 0%)
  adding: customer_behavior_model/1/ (stored 0%)
  adding: customer_behavior_model/1/saved_model.pb (deflated 88%)
  adding: customer_behavior_model/1/variables/ (stored 0%)
  adding: customer_behavior_model/1/variables/variables.index (deflated 64%)
  adding: customer_behavior_model/1/variables/variables.data-00000-of-00001 (deflated 54%)
  adding: customer_behavior_model/1/assets/ (stored 0%)


In [48]:
from google.colab import files
files.download('customer.zip')